# Weekly Challenge 01

*Original URL* https://community.alteryx.com/t5/Weekly-Challenge/Challenge-1-Join-to-Range/td-p/36621 and [**My Alteryx Approach**](https://github.com/dsmdavid/Alteryx-Weekly-Challenge/tree/master/submitted/sub_Challenge%2301)

## Brief
A company in Australia has source data which is made up of a series of postal codes (eg. 2000, 2001, 2002 etc.) amongst some other data fields. They have a separate reference table which contains postcode ranges (eg. 2000 to 2002) which they would like to use to match/filter their main data.


Each Customer Record needs to be joined to the Lookup table based on a Postal Area Ranged region. Then finally summarize the customer data by Region, Sales Rep, and Responder, then a count of customers.

 

Check and see what the result should look like by looking at the data labeled 'Output'.  Your mission is to take the input files and blend them so your result matches the output shown.  Good luck!


In [1]:
import pandas as pd
import numpy as np

# Explore

#### Read files

In [2]:
df_range = pd.read_csv("./01_files/range.csv")
df_customers = pd.read_csv("./01_files/customer.csv")

In [3]:
#display range
df_range.head()

,Range,Region,Sales Rep,Expect Revenue
0,2000-2019,R1,John,1000000
1,2020-2039,R2,Ted,3245234
2,2040-2059,R3,Nick,456654
3,2060-2079,R4,Mike,234545
4,2080-2100,R5,Paul,1232345


In [4]:
#display customers structure:
df_customers.head(5)

,Customer ID,Store Number,Customer Segment,Responder,Postal Area
0,2,100,Corporate,No,2086
1,3,100,Corporate,No,2051
2,5,100,Home Office,No,2077
3,6,106,Home Office,No,2004
4,8,101,Home Office,No,2010


In [5]:
df_customers.tail(5)

,Customer ID,Store Number,Customer Segment,Responder,Postal Area
2673,3403,107,Consumer,No,2044
2674,3390,101,Corporate,No,2053
2675,3393,106,Consumer,No,2022
2676,3391,105,Corporate,No,2093
2677,3402,100,Consumer,No,2040


## Approach I want to follow:
1. Create a function that assigns a postal code to a range.
2. Apply the function to the customers table.
3. Join and summarize.

In [6]:
#create columns with min & max for each range:
df_range['min'] = df_range['Range'].apply(lambda x: int(x.split('-')[0]))
df_range['max'] = df_range['Range'].apply(lambda x: int(x.split('-')[1]))

In [7]:
df_range

,Range,Region,Sales Rep,Expect Revenue,min,max
0,2000-2019,R1,John,1000000,2000,2019
1,2020-2039,R2,Ted,3245234,2020,2039
2,2040-2059,R3,Nick,456654,2040,2059
3,2060-2079,R4,Mike,234545,2060,2079
4,2080-2100,R5,Paul,1232345,2080,2100


In [8]:
def assignRange(postcode):
    '''Checks the postcode against the range.
    Probably it would have been better to create first a 2000-2100 list to use as a master --i.e. "generate rows".'''
    postcode = int(postcode)
    for x in df_range.index:
        if postcode >= df_range.ix[x]['min'] and postcode <= df_range.ix[x]['max']:
            return x

In [9]:
df_customers['Range_index'] = df_customers['Postal Area'].apply(assignRange)

df_customers.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


,Customer ID,Store Number,Customer Segment,Responder,Postal Area,Range_index
0,2,100,Corporate,No,2086,4
1,3,100,Corporate,No,2051,2
2,5,100,Home Office,No,2077,3
3,6,106,Home Office,No,2004,0
4,8,101,Home Office,No,2010,0


In [10]:
df_flat = pd.merge(df_customers, df_range, how = 'inner', left_on='Range_index', right_index=True)

df_flat.head()

,Customer ID,Store Number,Customer Segment,Responder,Postal Area,Range_index,Range,Region,Sales Rep,Expect Revenue,min,max
0,2,100,Corporate,No,2086,4,2080-2100,R5,Paul,1232345,2080,2100
12,23,107,Corporate,Yes,2095,4,2080-2100,R5,Paul,1232345,2080,2100
20,43,104,Consumer,No,2089,4,2080-2100,R5,Paul,1232345,2080,2100
23,52,103,Corporate,No,2089,4,2080-2100,R5,Paul,1232345,2080,2100
25,55,101,Home Office,No,2095,4,2080-2100,R5,Paul,1232345,2080,2100


In [11]:
df_flat.groupby(by=['Region','Sales Rep','Responder']).count()['Customer ID']

Region  Sales Rep  Responder
R1      John       No           476
                   Yes           76
R2      Ted        No           415
                   Yes           87
R3      Nick       No           493
                   Yes           92
R4      Mike       No           430
                   Yes           82
R5      Paul       No           434
                   Yes           93
Name: Customer ID, dtype: int64